In [ ]:
import sys
from pathlib import Path

NOTEBOOK_DIR = Path().resolve()
SRC_DIR = NOTEBOOK_DIR.parent / 'src'
if str(SRC_DIR) not in sys.path:
    sys.path.append(str(SRC_DIR))

from config import CONFIG
from train import train
from inference import generate_samples

In [ ]:
metrics = train(CONFIG)
metrics

### Interpret the metrics
- `train_loss` captures the MSE between predicted and true noise; expect a gradual decline.
- `val_loss` helps detect overfitting or divergence when experimenting with architectures.
- Convert to a pandas DataFrame for quick plotting and experiment comparison.
- Pair the metrics JSON with checkpoints for reproducibility.

In [ ]:
samples = generate_samples(CONFIG, num_images=16, output_path=CONFIG.artifact_dir / 'notebook_samples.png')
samples.shape

### Next experiments
- Enable mixed precision and benchmark speed gains.
- Reduce `CONFIG.sample_steps` to see how few steps still produce recognisable digits.
- Use TensorBoard callbacks to stream metrics during training.
- Compare generated grids with the PyTorch notebook outputs.